# Typical NLP-heavy problems

* *Classification*: Classifying documents into particular categories (giving labels).
* *Regression*: Predict numerical values
* *Clustering*: Separating documents into non- overlapping subsets.
* *Ranking*: For a given input, rank documents according to some metric. 
* *Association rule mining*: Infer likely associations patterns in data.
* *Structured output*: Bounding boxes, parse trees, etc.
* *Sequence-to-sequence*: For a given input/source text, generate output annotations or another text.

To address many of the challenges above, several important things has to be clarified beforehand:

1. Set the research/exploration goal (e.g., how heavy will be the earthquake on a given day).
2. Make a hypothesis ( e.g., strength of the earthquake is an informative signal).
3. Collect the data (e.g., collect historical strength of the quakes on eachday).
4. Test the hypothesis (e.g., train a model using the data)
5. Analyze the results (e.g., are results better than existing systems).
6. Reach a conclusion (e.g., the model should be used or not because of A, B, C).
7. Refine hypothesis and repeat. (e.g., time of the year could be a useful signal).

Below is the table of the approaches that can be applied to the problems/applications above.

| Type                   | Input   | Clarifications|
|------------------------|---------|---------------|
|   Rule-based           |Explicit linguistic patterns, <br>lexicons, etc.| Such an approach always have predefined behaviour and usually can't generalize usually. |
|   Supervised           | Training examples: typically <br>tuples of the form <br>(features, label) | Here input features are usually some vectorized, transformed, normalized input representation.|
|   Semi-supervised or <br> Pseudo-relevance <br>feedback   | Same as in supervised base, <br>but also results of the prediction <br>(e.g. with greatest confidence) <br>are used as input pairs.   | Once we trained the system on the input (feature, label), we label unknown examples <br>with the model. If the confidence of the label is high (on that unseen example), <br>we add those examples to the input set and retrain the model. |
|   Distant supervision  | Same as for supervised learning, <br>but (feature, label) pairs does not <br> come from the annotation, but from <br>some heuristic-based annotation.   | Rather than annotating thousands of examples (documents, sentences, words, etc.), <br>we take a few examples of the class and try to generalize and match those <br>in the domain-specific corpus. Such systems first generate or extract examples that are <br> very similar or slight modifications of the labelled input examples. For example, if you need to <br> find all the sentences about Obama's marriage, you would search for all sentences that match <br> Michelle and Barack Obama. Furthermore, those examples could be used to find any sentence <br> about a marriage. Another example could be for a given list of movies, match all the sentences that have <br> movie names. Clearly, such heuristics result in noisy input sets.|
|   Unsupervised         | Unlabeled features   | The system is expected to find some dependencies and patterns without clearly stating <br>which patterns we are looking for. E.g., clustering of documents, topics extraction <br>from the documents, etc. |
|   Hybrid               | Varies for method <br>combinations   | Combines several approaches that are mentioned earlier for various purposes. |

# Evaluation Metrics

Depending on the problem you solve or the  data (balanced number of classes or not) different metrics
<br>might be more suitable.

* **Accuracy**
<br>
$ Accuracy = \frac{n_{correct}}{N}$, where $N$ is a total number of example that we were analysing,
<br>$n_{corrent}$ is the number of examples that we have guessed the label.

*a.k.a. Classification*

* **Precision**
<br>
In case of classification, and in particular, if the classes are unbalanced, Precision should be a better measure to check.
<br>
$ Precision = \frac{n_{correct\ class\ prediction}}{n_{class\ predictions}} = \frac{TP}{TP + FP}$,
where $TP, FP, FN, TN$ are explained [here](https://en.wikipedia.org/wiki/False_positives_and_false_negatives).

* **Precision@K**
<br>
Once your task is not simply to classify some examples, but e.g. rank them, a popular metric is P@K.
<br>Here you use K (typically, 1,3,5, 10, etc.) and compute precision for those K results in your ranking.
<br>For example, if you have a query for which you need to find similar documents,
<br>P@K would be computed for the top K documents that are returned for a query as described above for those K elements.

* **Recall**
<br>
Another very important concept in NLP, is Recall - which basically tell us how many of the class examples
<br>or positive examples (maybe documents), our system had managed to extract.
<br>
$ Precision = \frac{n_{correct\ class\ prediction}}{n_{positive\ class\ examples}} = \frac{TP}{TP + FN}$ $

* **F1**
<br>
[F1](https://en.wikipedia.org/wiki/F1_score) is the harmonic mean of the two above metrics.
<br>Usually used to compare different approaches when you are not optimizing for P and R in particular
<br>but rather overall performance.

*a.k.a. Clustering* 

* [**Silhouette coefficient, Modularity, etc**](https://en.wikipedia.org/wiki/Cluster_analysis).
<br>
Once we move from the classification problems, and focus on clustering of the documents, many things can be measured depending
<br>if you have labels or not.
<br>
The case where we do not have labels: 
  * For already proposed clustering of the input, *modularity* would measure the well nodes are assigned to the clusters.
  <br> In particular, we would estimate how our current assignment is different from the assumed random graph.
  * Silhouette coefficient estimates how average distance between objects in the same clusters differs
  <br>from the average distance of those objects to the other clusters.
  * [Davies–Bouldin index](https://en.wikipedia.org/wiki/Davies-Bouldin_index) measures the difference between inter- and intra-cluster similarity.

*a.k.a. Language models*

* **Perplexity**
<br>
Once we consider text generation tasks, where we typically do not have labelled examples, or multiple correct answers
<br>are possible, *perplexity* can be used to evaluate your model. tl;dr - Perplexity estimates how surprised the model
<br>is upon receiving an input, e.g., how the model is surprised that the next word after a current one
<br>"eat" is "me", or "meat" or whatever. Typically, the lower the perplexity the more information about
<br>the input the model has (no surprises). Another interpretation is that we compare our probability
<br>distribution to the fair die.

# Set up

In [1]:
#@title Data preparation and preprocessing { display-mode: "form" }
# You can download these data from the following link and store somewhere locally:

csv_file = 'https://www.figure-eight.com/wp-content/uploads/2016/03/socialmedia-disaster-tweets-DFE.csv'

#    Example on how the data cab be loaded here from the local file system. 
#    More options here: https://colab.research.google.com/notebook#fileId=/v2/external/notebooks/io.ipynb&scrollTo=vz-jH8T_Uk2c
#    Downloaded the input csv.

from google.colab import files
uploaded = files.upload()

import pandas as pd
with open('socialmedia-disaster-tweets-DFE.csv',
          mode = 'r',
          encoding = 'ascii',
          errors = 'ignore'
         ) as csvfile:
  disasters_df = pd.read_csv(csvfile, header=0)

#    Golden examples
golden = disasters_df['_unit_state'] == "golden"
golden_positive = disasters_df['choose_one_gold'] == "Relevant"
golden_negative = disasters_df['choose_one_gold'] == "Not Relevant"

#    Annotated examples
finalized = disasters_df['_unit_state'] == "finalized"
confident = disasters_df['choose_one:confidence'] > 0.8
finalized_positive = disasters_df['choose_one'] == "Relevant"
finalized_negative = disasters_df['choose_one'] == "Not Relevant"

clean_confident_entries = disasters_df[
    golden | (finalized & finalized_positive & confident) | 
    (finalized & finalized_negative & confident)]
#    Need to decide on tokenization - most of the time " " is a good guess.

# Imports
# Note: following nltk packages should be downloaded
import nltk
nltk.download('punkt')
from nltk import (
    sent_tokenize as splitter,
    wordpunct_tokenize as tokenizer
)

# Splits a string into sentences and words.
def tokenize(text):
  return [tokenizer(sentence) for sentence in splitter(text)]

# In this exercise we do not care about the sentences (if any),
# so let's flatten the list.
def flatten(nested_list):
  return [item for sublist in nested_list for item in sublist]

def tokenize_flatten_df(row, field):
  return flatten(tokenize(row[field]))

import re

# remove urls
def remove_urls(text):
  return re.sub(r"(https?\://)\S+", "", text)

# remove mentions (@name) completely
def remove_mentions(text):
  return re.sub(r"@[^:| ]+:? ?", "", text)

# remove "RT:", if the tweet contains it.
def remove_rt(text):
  if text.lower().startswith("rt:"):
    return text[3:].strip()
  return text
def remove_urls_mentions_rt_df(row, field):
  return remove_rt(remove_mentions(remove_urls(row[field])))

clean_confident_entries['text_cleaned_from_url_mentions_rt'] = \
    clean_confident_entries.apply(
        lambda row: remove_urls_mentions_rt_df (row, 'text'),
        axis=1)

clean_confident_entries['text_tokenized'] = \
    clean_confident_entries.apply(
        lambda row:
            tokenize_flatten_df (row, 'text_cleaned_from_url_mentions_rt'),
        axis=1)
def replace_hashtags_from_text(text):
  return re.sub(r"#+ ?", "", text)
# remove hashtags
def replace_hashtags_from_list(tokens_list):
  return [token for token in tokens_list if token != "#"]

# remove digits
def remove_digits(tokens_list):
  return [token for token in tokens_list 
                if not re.match(r"[-+]?\d+(\.[0-9]*)?$", token)]

# remove all tokens that contains non alpha numeric, punctuation
def remove_containing_non_alphanum(tokens_list):
  return [token for token in tokens_list if token.isalpha()]
# lowercase everything
def lowercase_list(tokens_list):
  return [token.lower() for token in tokens_list]
from nltk.corpus import stopwords
nltk.download('stopwords')
# remove stopwords
def remove_stopwords(tokens_list):
  return [token for token in tokens_list
                if not token in stopwords.words(u'english')]
# Iterates over the elements of the list with tokens and performs cleanup.
def clean_tokens(row, field):
  return replace_hashtags_from_list(
            remove_digits(
                remove_containing_non_alphanum(
                    lowercase_list(remove_stopwords(row[field])))))

clean_confident_entries['text_tokenized_cleaned'] = \
    clean_confident_entries.apply(
        lambda row:
            clean_tokens (row, 'text_tokenized'),
        axis=1)
nltk.download('wordnet')
porter_stemmer = nltk.PorterStemmer()
lancaster_stemmer = nltk.LancasterStemmer()
snowball_stemmer = nltk.SnowballStemmer(u'english')
lemmatizer = nltk.WordNetLemmatizer()
lemmatizer.stem = lemmatizer.lemmatize
normalizers = [
    ('porter_stemmer', porter_stemmer),
    ('lancaster_stemmer', lancaster_stemmer),
    ('snowball_stemmer', snowball_stemmer),
    ('wordnet_lemmatizer', lemmatizer)
]
nltk.download('averaged_perceptron_tagger')
tagged = nltk.pos_tag(clean_confident_entries['text_tokenized'][0])
print (tagged)

!pip install --upgrade gensim

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

#    Note: it might take several minutes. Be patient!
from gensim.scripts.glove2word2vec import glove2word2vec

glove2word2vec(glove_input_file='glove.6B.50d.txt',
               word2vec_output_file="gensim_glove_vectors.txt")

from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt")

# Compute vector representation for each document in the collection.
# Term frequency
# TFIDF

from collections import defaultdict, Counter
import math

class TermDocumentCounts:
  def __init__(self):
    # Counters of all the words in the corpus
    self.total_word_counts = Counter()
    self.total_number_of_words = 0
    self.term_count_per_document = defaultdict(Counter)
    self.number_of_words_per_document = defaultdict(int)
    self.number_of_document = 0
    self.df = defaultdict(int)
    
  def update(self, document_id, tokens):
    self.number_of_document += 1
    num_tokens = len(tokens)
    self.total_word_counts.update(tokens)
    self.total_number_of_words += num_tokens
    self.term_count_per_document[document_id].update(tokens)
    self.number_of_words_per_document[document_id] += num_tokens
    for token in set(tokens):
      self.df[token] += 1
  
  def most_common_word_in_document(self, document_id, top_n = None):
    return self.term_count_per_document[document_id].most_common(top_n)
  
  def _compute_tfidf_for_word(self, document_id, word):
        tf = self.term_count_per_document[document_id][word]
        idf = math.log(self.number_of_document / self.df[word], 10)
        return tf * idf
  
  # Returns the list of words ranked accoring to TFIDF score.
  def ranked_document_words_tfidf(self, document_id, top_n=None):
        tfidfs = [
            (word, self._compute_tfidf_for_word(document_id, word))
              for word in self.term_count_per_document[document_id].keys()
        ]
        tfidfs.sort(key=lambda x: x[1], reverse=True)
        if not top_n:
            top_n = len(tfidfs)
        return tfidfs[:top_n]
      
  # Returns the list of words ranked accoring to max conditional probability
  # change.
  def ranked_document_words_conditional_probability(self, document_id,
                                                    top_n = None):
    word_posterior = [(word, self.compute_posterios(document_id, word)) \
              for word, count in \
                self.term_count_per_document[document_id].items()]
    word_prior = {word: self.compute_priors(word) for word, _ in word_posterior}
    conditional_probability = sorted(
        [(word, (math.log((probability / word_prior[word]), 2))) \
            for word, probability in word_posterior],
        key=lambda x: x[1],
        reverse=True
    )
    if not top_n:
        top_n = len(conditional_probability)
    return conditional_probability[:top_n]  
      
  # Computes posterior word distribution over given document.
  def compute_posterios(self, document_id, word):
    return self.term_count_per_document[document_id][word] \
              / self.number_of_words_per_document[document_id]
  
  # Computes prior word probability distribution
  def compute_priors(self, word):
    return self.total_word_counts[word] / self.total_number_of_words
    
corpus_counts = TermDocumentCounts()
for document_id, row in clean_confident_entries.iterrows():
  corpus_counts.update(document_id, row['text_tokenized_cleaned'])

Saving socialmedia-disaster-tweets-DFE.csv to socialmedia-disaster-tweets-DFE (1).csv
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[('Just', 'RB'), ('happened', 'VBD'), ('a', 'DT'), ('terrible', 'JJ'), ('car', 'NN'), ('crash', 'NN')]
     |████████████████████████████████| 24.2MB 1.2MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
--2019-07-23 11:34:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-07-23 11:34:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanfo

# Documents representation

Let's finally impose some vectorized representation on our documents so that the machine and math could easily operate over it.

### Document representation as term counts (Bag-Of-Words, or BOW)

In [ ]:
#    Now we need to convert our documents to the common representation.
#    You can do it manually, just for fun, or we can already use some libs.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer()

corpus = []
for document_id, row in clean_confident_entries.iterrows():
  corpus.append(" ".join(row['text_tokenized_cleaned']))
  
document_term_matrix = vectorizer.fit_transform(corpus).toarray()

In [5]:
document_term_matrix[2]

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
vectorizer.get_feature_names()[-10:]

['ziuw',
 'zombie',
 'zone',
 'zones',
 'zoom',
 'zouma',
 'zrnf',
 'zss',
 'zuma',
 'zumiez']

## Representation

* We need to represent our data somehow
  * Bag-of-words
  <br>
  The concept comes from the fact, that we could represent our input as a set of tokens or words without preserving any notion of order,
  <br> just like items in the bag. In such cases, we assume that each document is simply a set of words and each
  <br> word could appear several or zero times in a bag.
  <br> In order to somehow presenve the order in the documents, n-grams (n consequitive words/tokens in the document) could be introduced.
  <br> Though being rather powerful, number of features/dimentions grow exponentially.
  * Document-term frequency
  <br>
  Similar to the BOW model though instead of 0s and 1s for each word that appear in the document,
  <br> we would place TF or TFIDF score of the word in the document.
  * Semantic Vector space representation
  <br>
  Previous representation is rather simple but very very high dimentional, as usually number of tokens in the "bag" could be 
  <br> around 1M (despite them being present or not in the document). 
  <br> As you could have seen in the ML course, we could reduce the dimentionality by somehow converting our 
  <br> 1M dimentional document representation to let's say 300. 
  <br> Such transformation can be both done for words (special case of the document with a single 1 on the word position and 0s elsewhere),
  <br> combination of words, sentences, paragraphs, and whole documents.

### Feature weighting and noise removal

Apart from the general stopword removal that you might decide to do, several other techniques exist.
<br>Those could be used for, first, cleaning the overall corpus from too frequent/too sparse tokens.
<br>And, second, for potentially better representations of feature weights (rather thatn 0 and 1).

Note: would work better for BOW representation, but might work worse for sequence-to-sequence tasks.

*Weighting tokens/features*:

* *TF-IDF*
<br>
Term frequency * inverse document frequency. The intuition here is to preserve all words in the dataset in each document,
<br>however, **correct** their frequencies with word specificity, i.e., the more specific word is (if the word appear in only several
<br>documents it might be quite specific) the more we would want to promote word's score.
<br>One way to compute this specificity is to inverse docuemtn frequency. This way, if the term is very frequent accross documents -
<br>we would have to divide the term frequency with a high value, and vice versa.
<br>Note: here still the more frequent the word is in a document, the more likely it is to be important, though it might be slightly degraded if it is not specific.

* Filter words with the highest difference between *conditional probability* of a word within a document vs. a word within the whole corpus
<br>
For conditional probability, we do not care that much about the word frequency in a document, but rather how much its
<br>probability changes in a given document sample with respect to the overall probability in the entire dataset.
<br>
As a result, conditional probability might score higher more specific (but less frequent) words.

*Removing noise*:

* Filter out ones with the *highest Document Frequency or part of the probability mass*
<br>
You might have heard, that natural language vocabularies exhibit frequency distribution similar to [Zipf law](https://en.wikipedia.org/wiki/Zipfs_law)
<br>(subclass of the long-tail distribution where i_-frequent word would be twice as likely to appear in the corpus is _i+1_-frequent word).
<br>Though zipf allow more relaxed behaviour, i.e., a lot of tokens have low frequency and few have high frequency.

* *Background-foreground overlap idea*
<br>
The idea behind this methods is to compare frequency distributions of two independent datasets and diminish the intersection.
<br>This allows to deemphasize the effect of the high frequency noise. In more details, we take frequency distribution of one dataset,
<br>then same for some other totally independent (unrelated) dataset, extract top-n words in each distribution, and remove any items in the intersection.

In [ ]:
#@title Document Frequency Helper Functionality
# Compute vector representation for each document in the collection.
# Term frequency
# TFIDF

from collections import defaultdict, Counter
import math

class TermDocumentCounts:
  def __init__(self):
    # Counters of all the words in the corpus
    self.total_word_counts = Counter()
    self.total_number_of_words = 0
    self.term_count_per_document = defaultdict(Counter)
    self.number_of_words_per_document = defaultdict(int)
    self.number_of_document = 0
    self.df = defaultdict(int)
    
  def update(self, document_id, tokens):
    self.number_of_document += 1
    num_tokens = len(tokens)
    self.total_word_counts.update(tokens)
    self.total_number_of_words += num_tokens
    self.term_count_per_document[document_id].update(tokens)
    self.number_of_words_per_document[document_id] += num_tokens
    for token in set(tokens):
      self.df[token] += 1
  
  def most_common_word_in_document(self, document_id, top_n = None):
    return self.term_count_per_document[document_id].most_common(top_n)
  
  def _compute_tfidf_for_word(self, document_id, word):
        tf = self.term_count_per_document[document_id][word]
        idf = math.log(self.number_of_document / self.df[word], 10)
        return tf * idf
  
  # Returns the list of words ranked accoring to TFIDF score.
  def ranked_document_words_tfidf(self, document_id, top_n=None):
        tfidfs = [
            (word, self._compute_tfidf_for_word(document_id, word))
              for word in self.term_count_per_document[document_id].keys()
        ]
        tfidfs.sort(key=lambda x: x[1], reverse=True)
        if not top_n:
            top_n = len(tfidfs)
        return tfidfs[:top_n]
      
  # Returns the list of words ranked accoring to max conditional probability
  # change.
  def ranked_document_words_conditional_probability(self, document_id,
                                                    top_n = None):
    word_posterior = [(word, self.compute_posterios(document_id, word)) \
              for word, count in \
                self.term_count_per_document[document_id].items()]
    word_prior = {word: self.compute_priors(word) for word, _ in word_posterior}
    conditional_probability = sorted(
        [(word, (math.log((probability / word_prior[word]), 2))) \
            for word, probability in word_posterior],
        key=lambda x: x[1],
        reverse=True
    )
    if not top_n:
        top_n = len(conditional_probability)
    return conditional_probability[:top_n]  
      
  # Computes posterior word distribution over given document.
  def compute_posterios(self, document_id, word):
    return self.term_count_per_document[document_id][word] \
              / self.number_of_words_per_document[document_id]
  
  # Computes prior word probability distribution
  def compute_priors(self, word):
    return self.total_word_counts[word] / self.total_number_of_words
    

In [ ]:
corpus_counts = TermDocumentCounts()
for document_id, row in clean_confident_entries.iterrows():
  corpus_counts.update(document_id, row['text_tokenized_cleaned'])

What see what are words that are common in some of the documents.

In [14]:
for document_id in range(2):
  print ("\nDocument", document_id + 1,
         "with top 5 important words by tfidf or condiotional probability.")
  print (clean_confident_entries['text_tokenized'][document_id])
  print ("TFIDF", "\t\t\t\t\t", "Condifional probability")
  for word_tfidf, word_conditional in zip(
      corpus_counts.ranked_document_words_tfidf(document_id, 5),
      corpus_counts.ranked_document_words_conditional_probability(document_id,
                                                                  5)):
    print (word_tfidf, "\t", word_conditional)
for document_id in range(1001,1003):
  print ("\nDocument", document_id + 1, "with top 5 important words.")
  print (clean_confident_entries['text_tokenized'][document_id])
  print ("TFIDF", "\t\t\t\t\t", "Condifional probability")
  for word_tfidf, word_conditional in zip(
      corpus_counts.ranked_document_words_tfidf(document_id, 5),
      corpus_counts.ranked_document_words_conditional_probability(document_id,
                                                                  5)):
    print (word_tfidf, "\t\t", word_conditional)


Document 1 with top 5 important words by tfidf or condiotional probability.
['Just', 'happened', 'a', 'terrible', 'car', 'crash']
TFIDF 					 Condifional probability
('terrible', 2.966610986681934) 	 ('terrible', 10.79125593263882)
('happened', 2.697765674389354) 	 ('happened', 9.898171136555332)
('just', 2.033557776312547) 	 ('just', 7.691720259087907)
('car', 1.8874297406343095) 	 ('car', 7.122877423730027)
('crash', 1.7905197276262528) 	 ('crash', 6.778431892281237)

Document 2 with top 5 important words by tfidf or condiotional probability.
['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', '#', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all']
TFIDF 					 Condifional probability
('deeds', 3.811709026696191) 	 ('deeds', 12.920538949583786)
('forgive', 3.3345877719765284) 	 ('forgive', 11.33557644886263)
('allah', 3.1127390223601723) 	 ('allah', 10.598610854696425)
('our', 2.581260105317917) 	 ('our', 8.833076108333447)
('reason', 2.581260105317917) 	 ('reason', 8.833076108

## Feature extraction

Before coding, check modules of [scikit-learn](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction) and more examples [here](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction). 

*Potential features*:

* Words
* N-grams
* Character N-gram
* Skip-gram
* Part-of-Speech (POS)

*Potential values*:

* TF/Count Vectors ([CountVectorizer scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html),  ([HashingVectorizer scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html)))
* TF-IDF ([TFIDFVectorizer scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) or [TFIDFTransformer scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html))


## Preserving Word Order

Depending what is your particular problem, you might need to care about the order of words in your input data.

* This could happen either on the data preprocessing step, where you would encode the order as part of the feature input,\
<br>e.g., n-grams, specific features that correspond to the positions of the words.
<br>Further all those features are sent to the particular methods of your choice.
* If you do not want and can't polute the input with those additional information, you might need to reply on the methods that
<br>implicitly encode the order in the data as they read the input.
  * Hiden Markov Models ([HMM on wiki](https://en.wikipedia.org/wiki/Hidden_Markov_model), [HMM Fundamentals](http://cs229.stanford.edu/section/cs229-hmm.pdf)), Conditional Random Fields ([CRF](https://en.wikipedia.org/wiki/Conditional_random_field)), Reccurrent neural networks, etc.


---

# Document Classification

---

# Simple Classification by Lexical Rules


 $\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ $ $\Rightarrow_{YES}$  Relevant

Text Document $\ \ \ \Rightarrow \ \ \ $ Are there any relevant terms present?

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ $ $\Rightarrow_{NO}$ Rot_Relevant

## Building lexicons: PMI and Relatedness (Strength of Association)

One classic way to document classification is to classify texts based on the presence of terms from  either existing or heuristically built lexicons.

Pointwise Mutual Information (or PMI) can be a great estimator of the informativeness or relevance weight of words. It was originally introduced to measure how strongly two words are associated, that is how often they go together among their occurrences.

In the context of classification, PMI measures the relatedness between a term $t$ and a certain class $c$. It is defined as follows:

$PMI(t,c) = \log_2\frac{P(t, c)}{P(t)P(c)} = \log_2\frac{P(t|c)}{P(t)}$, where $P(t,c)$ is a joint probability of $t$ and $c$, and $P(t)$, $P(c)$ are the marginal probabilities of $t$ and $c$.

Another potential reason to explore the most important or relevant keywords in the dataset is to determine whether there are some dependencies and correlations in the data. Thus, confirming potential benefit of the machine intelligence :)

Note: simple correlation measure could be also checked prior to PMI.


In [ ]:
import math
def compute_terms_pmi(inclass_corpus_counts, total_corpus_counts):
  PMI_dict = defaultdict(float)
  for word, inclass_count in inclass_corpus_counts.total_word_counts.items():
    inclass_term_probability = float(inclass_count / inclass_corpus_counts.total_number_of_words)
    marginal_term_probability = \
      float(total_corpus_counts.total_word_counts[word] / \
            total_corpus_counts.total_number_of_words)
    PMI_dict[word] = math.log(float(inclass_term_probability / \
                              marginal_term_probability), 2)
  return PMI_dict

In [ ]:
# We compute PMI of all words in the corpus to the positive 'relevant' class.
positive_corpus_counts = TermDocumentCounts()
for document_id, row in clean_confident_entries.iterrows():
  if row['choose_one'] == "Relevant" or \
        row['choose_one_gold'] == "Relevant":
      positive_corpus_counts.update(document_id, row['text_tokenized_cleaned'])
      
PMI_positive = compute_terms_pmi(positive_corpus_counts, corpus_counts)

In [17]:
for related_words in sorted(PMI_positive.items(), key=lambda x:x[1], reverse=True)[:10]:
  print (related_words)

('deeds', 1.02804356582442)
('geese', 1.02804356582442)
('fleeing', 1.02804356582442)
('ronge', 1.02804356582442)
('sask', 1.02804356582442)
('residents', 1.02804356582442)
('shelter', 1.02804356582442)
('notified', 1.02804356582442)
('officers', 1.02804356582442)
('orders', 1.02804356582442)


In [20]:
# Print occurrences for particular word:
word = 'orders'
print('Term:', word)
print('Appeared in positive class:', positive_corpus_counts.total_word_counts[word], 'times')
print('Appeared overall:', corpus_counts.total_word_counts[word], 'times')
print('PMI to positive class:', PMI_positive[word])

Term: orders
Appeared in positive class: 8 times
Appeared overall: 8 times
PMI to positive class: 1.02804356582442


However, the above measure does not tell us anything if a term with a high PMI for a given class has also a very high score for a "not-class". A better metric would be to compute a difference between a term's PMIs to related texts and non-related texts. This approach was used for [building lexicons of relevant words for data collection](http://www.aaai.org/ocs/index.php/ICWSM/ICWSM14/paper/download/8091/8138/) and for [bulding sentiment lexicons](https://arxiv.org/pdf/cs/0212032.pdf). 

<br>
The score (called Relatedness or Strength of Association) will be:
<br>
$$relatedness_{PMI}(t) = PMI(t, related) - PMI(t, not\ related) = $$ $$\log_2\frac{p(t|related)}{p(t|not\ related)}$$ where
<br>
$$p(t|related)= \frac{count(t\ in\ related)}{count(t\ in\ related) + count(not\ t\ in\ related)}$$
<br>
$$p(t|not\ related)= \frac{count(t\ in\ not\ related)}{count(t\ in\ not\ related) + count(not\ t\ in\ not\ related)}$$

In [ ]:
# We compute PMI of all words in the corpus to the negative 'not relevant' class.
negative_corpus_counts = TermDocumentCounts()
for document_id, row in clean_confident_entries.iterrows():
  if row['choose_one'] == "Not Relevant" or \
        row['choose_one_gold'] == "Not Relevant":
      negative_corpus_counts.update(document_id, row['text_tokenized_cleaned'])
      
PMI_negative = compute_terms_pmi(negative_corpus_counts, corpus_counts)

In [ ]:
# Computing PMI relatedness score for all terms in the corpus.
# Terms occurring only in one class will have zero score for another class.
PMI_relatedness = defaultdict(float)
for word in corpus_counts.df.keys():
  pmi_word_pos = PMI_positive[word] if word in PMI_positive else 0.0
  pmi_word_neg = PMI_negative[word] if word in PMI_negative else 0.0
  PMI_relatedness[word] = pmi_word_pos - pmi_word_neg
  
# Note: If you'd like, you can try to implement count-based version as described in the
# formulas above.

In [23]:
for related_words in sorted(PMI_relatedness.items(), key=lambda x:x[1], reverse=True)[:10]:
  print (related_words)

('suicide', 7.285146784187203)
('bombing', 6.4313675250382465)
('homes', 6.265781459320271)
('families', 5.641290594412478)
('spill', 5.641290594412478)
('warning', 5.61091694536896)
('outbreak', 5.579890049748334)
('california', 5.51575971232862)
('wildfire', 5.4313675250382465)
('anniversary', 5.413880098309407)


In [24]:
for nonrelated_words in sorted(PMI_relatedness.items(), key=lambda x:x[1], reverse=False)[:10]:
  print (nonrelated_words)

('screamed', -4.943671906308678)
('blew', -4.897868216695554)
('fan', -4.801652901436251)
('panic', -4.751026828366283)
('crushed', -4.698559408472147)
('things', -4.64411162444977)
('pick', -4.467233862365691)
('quiz', -4.335989329087439)
('finally', -4.265600001196041)
('hat', -4.191599419752264)


In [25]:
# Print occurrences for particular word:
word = 'canada'
print('Term:', word)
print('Appeared in positive class:', positive_corpus_counts.total_word_counts[word], 'times')
print('Appeared in negative class:', negative_corpus_counts.total_word_counts[word], 'times')
print('Appeared overall:', corpus_counts.total_word_counts[word], 'times')
print('PMI to positive class:', PMI_positive[word])
print('PMI to negative class:', PMI_negative[word])
print('PMI relatedness score :', PMI_relatedness[word])

Term: canada
Appeared in positive class: 10 times
Appeared in negative class: 4 times
Appeared overall: 14 times
PMI to positive class: 0.5426167386541784
PMI to negative class: -0.835639449924506
PMI relatedness score : 1.3782561885786844


In [26]:
# Examples of texts with a particular word.
word = 'canada'
limit_to_print = 5
current_print = 0

for document_id, row in clean_confident_entries.iterrows():
  if word in row['text_tokenized_cleaned']:
    print(row['text_tokenized_cleaned'])
    print(row['choose_one'], row['choose_one_gold'])
    current_print += 1
    if (current_print >= limit_to_print):
      break

['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada']
Relevant Relevant
['idgaf', 'tough', 'canada', 'north', 'philly', 'meek', 'acting', 'like', 'bitch', 'amp', 'drake', 'body', 'bagging', 'ass', 'tracks']
Not Relevant nan
['the', 'cryptic', 'words', 'guided', 'pilots', 'hiroshima', 'bombing', 'mission', 'canada']
Relevant nan
['a', 'decade', 'long', 'billion', 'dollar', 'deluge', 'messages', 'government', 'canada', 'harperslegacy']
Not Relevant nan
['dead', 'injured', 'displaced', 'syria', 'visit', 'us', 'canada']
Relevant nan


In [27]:
# Explore terms with PMI relatedness around 0 (no strong relatedness to any).
# They are candidates for removal from the lexicon.
for weak_words in sorted(PMI_relatedness.items(), key=lambda x:abs(x[1]), reverse=False)[:10]:
  print (weak_words)

('another', 0.003860673797185999)
('say', 0.017853945876685998)
('movie', -0.031134747559017736)
('man', -0.03678131070015989)
('order', -0.050587110225190114)
('live', -0.05470321869742224)
('days', 0.056328093691321565)
('london', 0.056328093691321565)
('consider', 0.056328093691321565)
('park', 0.056328093691321565)


Note: The same techniques can be used to assign term feature weights and for feature selection.

Note: PMI is biased towards inflating scores for low-occuring terms, it requires some smoothing of occurrences (e.g. by simply adding 1 to all counts: [Add-1 or Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)) and ideally removal of such low-occuring terms.

# Document Classification with Machine-Learning Models

## General description

Classification is a prediction problem where by a given (labelled) set of data, e.g., data representation (features) and each entry class.
<br>
There can be two or multiple classes: 
* (1) positive/negative - binary classification;
* (2) multiple classes - multi-class classification problem.

The latter can be represented via several binary classifications.

Let's try to get some indepth understanding of the two important classification methods that are used in practice, easy and fast.

### Notations

Let's denote $\dot X$ -  our input set (observation set), and $\dot y$ - our output set.

Note: It is possible that no assumptions are made on the $\dot X$ and $\dot y$, however,
<br>
in this class we assume that $\dot y \in \{c_1, \dots c_k\}$ is a finite set and it contains 
<br>
$k$ classes (labels).
<br>
In our use case, $\dot X$ will be a set of documents (as we have seen in the previous exercises), and
<br>
$\dot y$ is a set of document categories. The classification goal is to assign a category to each document.

More notations:

* $X$ is a random variable taking values in the same space as entries of $\dot X$.
* $y$ is a random variable taking values in the same space as entries of $\dot y$.

* $P(y)$ denotes a probability associated with the event $Y = \dot y$, and similarly for joint and conditional probabilities.

Probability recap:

$P(x, y) = P(y|x) P(x) = P(x|y) P(y)$

### Model types

**Generative** (joint) classifiers place probabilities over both observed data and the hidden structure
<br>(a.k.a., generate the observed data from the hidden structure):
<br>
Example models: n-gram language models, Naive Bayes, Hidden Markov Models, Probabilistic context-free grammats, etc.
<br>
These models give joint probabilities $P(X,y)$ and try to maximize joint likelihood.

**Discriminative** (conditional) models take the data as given, and put a probability over hidden structure given the data.
<br>
Example models: Logistic regression, conditional loglinear or maximum entropy models, conditional random fields, SVM, perceptron, etc.
<br>
These models give conditional probabilities $P(y|X)$ and try to maximize conditional likelihood.
<br>Here, the observed data is taken as is and the output class is modelled only using conditional probability.

More details about these model types [here](https://web.stanford.edu/class/cs124/lec/Maximum_Entropy_Classifiers.pdf).


## <font color='gray'>Generative Classifiers: </font> [Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)

If we knew the true distribution $P(X, y)$, the best possible classifier (Bayes optimal) would be the one that predicts according to:
<br>
$ \hat y =  arg\,max_{y \in Y} P(y|x)$
<br>
$ = arg\,max_{y \in Y} \frac{P(x, y)}{P(x)} $
<br>
$ \hat = arg\,max_{y \in Y} P(x, y) $
<br>
$  = arg\,max_{y \in Y} P(y) P(x|y) $

Thus, we need to estimate the probability distributions $P(y)$ and $P(X|y)$ - class priors and conditional, respectively.

Assumption is that the data is generated according to following generative scenario (independently for each $m = 1, \dots M$).

1. A class $y_m \sim P(y)$ is drawn from the class prior distribution
2. An input $X_m \sim P(X|y = y_m)$ is drawn from the corresponding class conditional.

### Training and inference

To train the generative model, we need to estimate probabilities $P(y)$ and $P(X|y)$ using given dataset.

Once the training is done, we take new input and we need to make predictions according to:
<br>
$ \hat y = arg\,max_{y \in Y} \hat P(y) \hat P(x|y)$ using the estimated probabilities.

This step is called inference or decoding.

Two problems remain:

1. How we shoud define the distributions $\hat P(y)$ and $\hat P(X|y)$? (any independence assumptions etc.)
2. How to estimate the parameters of the distributions from the training data?

*Former*, is dependent on the application. Quite often, there is a natural decomposition of the input variables $X$ in $J$ components
<br>(like a document is decomposed to the words/features etc).
<br>
The **Naive Bayes** method makes the following assumptions: $X_1, \dots, X_J$ are conditionally independent given the class.
<br>Or
$$P(X|y) = \prod_{j=1}^{J} P (X_j | y)$$

As a result, such an assumption, greatly reduces the number of parameters to be estimated (degrees of freedom, dimentionalities) -
<br>from $O(\exp(J))$ to $O(J)$.
<br>Thus, computation become more efficient for large $J$ and the risk of overfitting also descreases.

*Latter* problem, can be solved using **maximum likelihood estimation**, which aims to maximize the probability of the training sample,
<br>assuming that each point was generated **independently** from selected distribution. Let's denote such probability as follows:
$$ P(Sample) = \prod_{m=1}^{M} P(x^m, y^m)$$
$$ = \prod_{m=1}^{M} P(y^m) \prod_{j=1}^{J} P(x_j^m|y^m)$$

### Example: Multinomial Naive Bayes for Document Classification

Let's say we need to classifiy documents: $X$ is the set of possible documents, and $y = \{i_1, \dots, y_k\}$ is a set of clases for those documents.
<br>Let $V = \{w_1, \dots, w_J\}$ be the vocabulary, i.e., set of words that appear in the documents.

As one of the document representation that we have seen, BOW can be applied here (order of words is ignored and thus each document is
<br>a collection of the words with their frequencies).

Interesting, such a representation - BOW - is equivalent to the Naive Bayes assumption with the multinomial model.
<br>
What does it mean, is that, we associate to each class a multinomial distribution, that ignores the word ordering,
<br>but takes into consideration the frequencies with which each word appears in a document.

In such scenario, each document is assumed to be generated as follows. 
1. Class y is generated according to $P(y)$.
2. $X$ is generated by sequentially picking words from $V$ (vocabulary) with replacement.
<br>
Each word is picked with probability $P(w_j|y)$.

For example, the probability to generate a document $x = w_{j1} \dots w_{jL}$ is
$$ P(x|y) = \prod_{l=1}^{L} P(w_{jl}|y) =  \prod_{j=1}^{J} P(w_{j}|y)^{n_j(x)}$$
, where L is the length of the docuemnt; $n_j(x)$ is the number of occurrences of word $w_j$ in document $x$.

Considering the independence assumption, we only need to estimate the following parameters:
<br>
$\hat P(y_1), \dots, \hat P(y_K)$, and $\hat P(w_j | y_k) $ for $j=1, \dots, J$ and $k = 1, \dots, K$.

Let's denote $J_k$ indices of the documents that are labeled with the class $k$.

Thus, *maximum likelihood estimation* prior and conditionals are the following:
<br>
$ \hat P (y_k) = \frac{|J_K|}{M}$ - relative frequencies of the classes
<br>
$ \hat P (w_j | y_k) = \frac{ \sum_{m \in J_k} n_j(x_m)}{ \sum_{i=1}^{J} \sum_{m \in J_k} n_i(x_m) }$ - relative frequencies of the words accross documents with that class.

Important, what will happen if during the prediction we encounter unseen words?
<br>
We can solve it using add-one smoothing technique.
<br>
$ \hat P (w_j | y_k) = \frac{ 1 + \sum_{m \in J_k} n_j(x_m)}{ V + \sum_{i=1}^{J} \sum_{m \in J_k} n_i(x_m) }$, where $V$ is the number of distinct words.


In [ ]:
#@title Multinomial Naive Bayes Implementation
import numpy as np

class MultinomialNaiveBayes:
    def __init__(self):
        self.trained = False
        self.likelihood = 0
        self.prior = 0
        self.smooth = True
        self.smooth_param = 1

    def train(self, x, y):
        # n_docs = no. of documents
        # n_words = no. of unique words
        n_docs, n_words = x.shape

        # classes = a list of possible classes
        classes = np.unique(y)
        # n_classes = no. of classes
        n_classes = np.unique(y).shape[0]

        # initialization of the prior and likelihood variables
        prior = np.zeros(n_classes)
        likelihood = np.zeros((n_words, n_classes))

        # We need to compute the values of the prior and likelihood parameters
        # and place them in the variables called "prior" and "likelihood".
        # Examples:
        # prior[0] is the prior probability of a document being of class 0
        # likelihood[4, 0] is the likelihood of the fifth feature/word being
        # active, given that the document is of class 0

        for i in range(n_classes):
            # docs_in_class = indices of documents in class i
            docs_in_class = np.nonzero(y == classes[i])
            # rior = fraction of documents with this class
            prior[i] = 1.0 * len(docs_in_class) / n_docs

            # word_count_in_class = count of word occurrences in documents
            # of class i
            word_count_in_class = np.sum(x[docs_in_class, :][0], axis=0)
            # total_words_in_class = total number of words in
            # documents of class i
            total_words_in_class = word_count_in_class.sum()
            if not self.smooth:
                # likelihood = count of occurrences of a word in a class
                likelihood[:, i] = word_count_in_class / total_words_in_class
            else:
                likelihood[:, i] = \
                    (word_count_in_class + self.smooth_param) \
                    / (total_words_in_class + self.smooth_param*n_words)

        # +1 account for the parameters of the prior estimation
        params = np.zeros((n_words+1, n_classes))
        for i in range(n_classes):
            params[0, i] = np.log(prior[i])
            params[1:, i] = np.nan_to_num(np.log(likelihood[:, i]))
        self.likelihood = likelihood
        self.prior = prior
        self.trained = True
        return params
      
    def get_label(self, x, w):
        # Computes the label for each data point
        scores = np.dot(x, w)
        return np.argmax(scores, axis=1).transpose()

    def test(self, x, w):
        # Classifies the points based on a weight vector.
        if not self.trained:
            raise ValueError("Model not trained. Cannot test")
            return 0
        x = self.add_intercept_term(x)
        return self.get_label(x, w)

    def add_intercept_term(self, x):
        # Adds a column of ones to estimate the intercept term for separation
        # boundary
        n_docs, n_features = x.shape
        intercept = np.ones([n_docs, 1])
        x = np.hstack((intercept, x))
        return x

    def evaluate(self, truth, predicted):
        correct = 0
        total = 0
        for i in range(len(truth)):
            if truth[i] == predicted[i]:
                correct += 1
            total += 1
        return correct / total

##  <font color='gray'> Discriminative classifiers: </font> [Logistic Regression](https://en.wikipedia.org/wiki/Multinomial_logistic_regression) (Maximum Entropy)

Classifiers that does not explicitly model $P(y)$ and $P(X|y)$ are called discriminative.
<br>
They are basically any function that maps objects $x \in X$ into classes $y \in Y$.

In such classifiers, input $x \in X$ is subject to a set of descriptions or measurements, which are called features.
<br>A feature is simply a real number that describes the value of some property of input.

*Feature mapping* is a mapping from the input $X$ to the *feature vector representation of x.*

Feature can be anything, like "a sentence contains word Trump", etc.

*Join feature mapping* is a collection of join input-output measurements, e.g., "a sentence contains word Trump and topic is election".

Linear classifiers are very popular in NLP applications. The classification decision in such models are made based on the rule:
<br>
$$ \hat y = arg\,max\, \lambda \cdot f(x, y) $$
where $\lambda$ - weight vector,
$f(x, y)$ - feature vector,
<br>
$\lambda \cdot f(x, y) = \sum_{d=1}^{D} \lambda_d f_d(x,y)$ - inner product between $\lambda$ and $f(x,y)$.

As a result, $f_d(x,y)$ has a weight $\lambda_d$, and for each class $y \in Y$,
<br>a score is computed as linear combination of all the weighted features.

If we decompose weight vector $\lambda = (\lambda_1, \dots, \lambda_K)$, we can have
<br>$\lambda \cdot f(x, y) == \lambda_k \cdot f(x)$ - a.k.a., each class gets its own weight vector and we only need
<br>a feature vectors that only looks at the input.

Prediction rule will be the following for such models:
$$ \hat y = arg\,max\,\lambda_k \cdot f(x) $$

Despite the representation details, exponential models (log-linear, maxent, logistic, Gibbs):
* Make a probability model from the linear combination of weight and features.
$$ P(y| X, \lambda)  = \frac{\exp \sum_d \lambda_d f_d(x, y)}{\sum_{k}\exp \sum_d \lambda_d f_d(x, y_k)}$$
Interesting, is that the gradient of the logarithm of the denominator - equals the feature expectation.
* Weights are the parameters of the probability model, combined via a soft max function.

Now, we estimate conditional log-likelihood:
$$ L(\lambda; Data) = \frac{1}{N} \log P_{\lambda} (y^1, \dots, y^N | x^1, \dots, x^N) $$
$$ = \frac{1}{N} \log \prod_{n=1}^{N} P_{\lambda} (y^n | x^n) $$
$$ = \frac{1}{N} \sum_{n=1}^{N} \log P_{\lambda} (y^n | x^n) $$

So $\lambda$ can be found as $arg\,max \, L(\lambda; Data)$, plus we can add some regularization here.

We won't have a closed form solution for the problem, thus, we follow the following standard procedure.
<br>Since the function is convex (-log), local optimum equals to the global one, thus, we differentiate it by our parameter and
<br>using some numerical method to find the parameters that optimizes our function.

We can use batch gradient method to optimize our problem.
$\lambda^{t+1} = \lambda^t - \mu_t \nabla_{\lambda} (-\log L(\lambda; Data))$

In [ ]:
#@title Perceptrone code
import numpy as np

class Perceptron:
    def __init__(self, nr_epochs=10, learning_rate=1, averaged=True):
        self.trained = False
        self.nr_epochs = nr_epochs
        self.learning_rate = learning_rate
        self.averaged = averaged
        self.params_per_round = []

    def train(self, x, y, seed=1):
        self.params_per_round = []
        x_orig = x[:, :]
        x = self.add_intercept_term(x)
        nr_x, nr_f = x.shape
        nr_c = np.unique(y).shape[0]
        w = np.zeros((nr_f, nr_c))
        for epoch_nr in range(self.nr_epochs):

            # use seed to generate permutation
            np.random.seed(seed)
            perm = np.random.permutation(nr_x)

            # change the seed so next epoch we don't get the same permutation
            seed += 1

            for nr in range(nr_x):
                # print "iter %i" %( epoch_nr * nr_x + nr)
                inst = perm[nr]
                y_hat = self.get_label(x[inst:inst+1, :], w)

                if y[inst:inst+1] != y_hat:
                    # Increase features of th e truth
                    w[:, y[inst:inst+1]] += \
                        self.learning_rate * x[inst:inst+1, :].transpose()

                    # Decrease features of the prediction
                    w[:, y_hat] += \
                        -1 * self.learning_rate * x[inst:inst+1, :].transpose()

            self.params_per_round.append(w.copy())
            self.trained = True
            y_pred = self.test(x_orig, w)
            acc = self.evaluate(y, y_pred)
            self.trained = False
            print("Rounds: %i Accuracy: %f" % (epoch_nr, acc))
        self.trained = True

        if self.averaged:
            new_w = 0
            for old_w in self.params_per_round:
                new_w += old_w
            new_w /= len(self.params_per_round)
            return new_w
        return w

    def get_label(self, x, w):
        # Computes the label for each data point
        scores = np.dot(x, w)
        return np.argmax(scores, axis=1).transpose()

    def test(self, x, w):
        # Classifies the points based on a weight vector.
        if not self.trained:
            raise ValueError("Model not trained. Cannot test")
            return 0
        x = self.add_intercept_term(x)
        return self.get_label(x, w)

    def add_intercept_term(self, x):
        # Adds a column of ones to estimate the intercept term for separation
        # boundary
        nr_x, nr_f = x.shape
        intercept = np.ones([nr_x, 1])
        x = np.hstack((intercept, x))
        return x

    def evaluate(self, truth, predicted):
        correct = 0
        total = 0
        for i in range(len(truth)):
            if truth[i] == predicted[i]:
                correct += 1
            total += 1
        return correct / total

In [ ]:
#@title Maximum Entropy Classifier (or Logistic Regression) Implementation
import numpy as np
import scipy.optimize.lbfgsb as opt2
from tensorflow.contrib.solvers.python.ops import util

class MaxEntBatch:

    def __init__(self, regularizer=1):
        self.trained = False
        self.parameters = 0
        self.regularizer = regularizer

    def train(self, x, y):
        x = self.add_intercept_term(x)
        n_docs, n_features = x.shape
        classes = np.unique(y)
        n_classes = classes.shape[0]
        # Add the bias feature
        init_parameters = np.zeros((n_features, n_classes), dtype=float)
        empirical_counts = np.zeros((n_features, n_classes))
        classes_idx = []
        for c_i, c in enumerate(classes):
            idx = np.nonzero(y == c)
            classes_idx.append(idx)
            empirical_counts[:, c_i] = np.sum(x[idx, :], axis = 0)[0]
        params = self.minimize_lbfgs(init_parameters, x, y, self.regularizer,
                                     empirical_counts, classes_idx, n_docs,
                                     n_features, n_classes)
        self.trained = True
        return params

    def minimize_lbfgs(self, parameters, x, y, sigma, empirical_counts,
                       classes_idx, n_docs, n_features, n_classes):
        parameters_flattened = parameters.reshape([n_features*n_classes],
                                                  order="F")
        result, _, d = opt2.fmin_l_bfgs_b(self.get_objective,
                                          parameters_flattened,
                                          args=[x, y, sigma, empirical_counts,
                                                classes_idx, n_docs, n_features,
                                                n_classes])
        return result.reshape([n_features, n_classes], order="F")

    # Obj =  -sum_(x,y) p(y|x) + sigma*||w||_2^2
    # Obj = \sum_(x,y) -w*f(x,y)+log(\sum_(y') exp(w*f(x,y')))+sigma*||w||_2^2
    def get_objective(self, parameters_flattened, x, y, sigma, empirical_counts,
                      classes_idx, n_docs, n_features, n_classes):
        parameters = parameters_flattened.reshape([n_features, n_classes],
                                                  order="F")
        # f(x,y).w
        scores = np.dot(x, parameters)
        # exp(f(x,y).w)
        exp_scores = np.exp(scores)
        # sum_y exp(f(x,y).w)
        z = exp_scores.sum(axis=1).reshape([n_docs, 1], order="F")
        # log sum_y exp(f(x,y).w)
        logz = np.log(z)
        sum_scores = 0
        for i, classes in enumerate(classes_idx):
            sum_scores += np.sum(scores[classes, i])

        objective = - sum_scores / n_docs \
                    + np.sum(logz) / n_docs \
                    + 0.5 * sigma * util.l2norm_squared(parameters)
        probs = exp_scores / z
        exp_feat = np.dot(x.transpose(), probs)
        grad = - empirical_counts / n_docs \
               + exp_feat / n_docs \
               + parameters * sigma 
        print("Objective = {0}".format(objective))
        return objective, grad.reshape([n_features * n_classes], order="F")
      
    def get_label(self, x, w):
        # Computes the label for each data point
        scores = np.dot(x, w)
        return np.argmax(scores, axis=1).transpose()

    def test(self, x, w):
        # Classifies the points based on a weight vector.
        if not self.trained:
            raise ValueError("Model not trained. Cannot test")
            return 0
        x = self.add_intercept_term(x)
        return self.get_label(x, w)

    def add_intercept_term(self, x):
        # Adds a column of ones to estimate the intercept term for separation
        # boundary
        nr_x, nr_f = x.shape
        intercept = np.ones([nr_x, 1])
        x = np.hstack((intercept, x))
        return x

    def evaluate(self, truth, predicted):
        correct = 0
        total = 0
        for i in range(len(truth)):
            if truth[i] == predicted[i]:
                correct += 1
            total += 1
        return correct / total

## Running various classifiers

In [ ]:
# Generating corpus of texts with corresponding golden labels.
import numpy as np

corpus = []
labels = np.zeros(clean_confident_entries.shape[0], dtype=int)
for i, (document_id, row) in enumerate(clean_confident_entries.iterrows()):
  corpus.append(" ".join(row['text_tokenized_cleaned']))
  if (row['choose_one'].lower() == "relevant" or \
        row['choose_one_gold'] == "Relevant"):
    labels[i] = 1

In [ ]:
# We split the data into train/tet to avoid overfitting. Another strategy would be to do cross-validation, as below.
from sklearn.model_selection import train_test_split

corpus_train, corpus_test, labels_train, labels_test = train_test_split(
   corpus, labels, test_size=0.33, random_state=1024)

In [ ]:
#@title Get feature representation of documents
#    You can do it manually, just for fun, or we can already use some libs.
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

vectorizer = CountVectorizer()

# We build document-term matrix for each dataset:
document_term_matrix_train = vectorizer.fit_transform(corpus_train).toarray()
document_term_matrix_test = vectorizer.transform(corpus_test).toarray()

In [ ]:
# Configuring evaluation function.
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def evaluate_classifier(classifier, X_train, X_test, y_train, y_test):
  classifier.fit(X_train, y_train)
  predicted_y_test = classifier.predict(X_test)
  print("Accuracy:", accuracy_score(y_test, predicted_y_test))
  report = classification_report(y_test, predicted_y_test)
  print(report)

In [35]:
#@title Run evaluation with MultinomialNB:
from sklearn.naive_bayes import MultinomialNB

evaluate_classifier(MultinomialNB(),
                    document_term_matrix_train, document_term_matrix_test,
                    labels_train, labels_test)

Accuracy: 0.8775700934579439
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1154
           1       0.89      0.84      0.86       986

    accuracy                           0.88      2140
   macro avg       0.88      0.88      0.88      2140
weighted avg       0.88      0.88      0.88      2140



In [36]:
#@title Run evaluation with Perceptron classifier:
from sklearn.linear_model import Perceptron
evaluate_classifier(Perceptron(),
                    document_term_matrix_train, document_term_matrix_test,
                    labels_train, labels_test)

Accuracy: 0.866822429906542
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      1154
           1       0.86      0.84      0.85       986

    accuracy                           0.87      2140
   macro avg       0.87      0.87      0.87      2140
weighted avg       0.87      0.87      0.87      2140



In [37]:
#@title Run evaluation with Logistic regression classifier:
from sklearn.linear_model import LogisticRegression
evaluate_classifier(LogisticRegression(),
                    document_term_matrix_train, document_term_matrix_test,
                    labels_train, labels_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.8855140186915887
              precision    recall  f1-score   support

           0       0.86      0.94      0.90      1154
           1       0.92      0.82      0.87       986

    accuracy                           0.89      2140
   macro avg       0.89      0.88      0.88      2140
weighted avg       0.89      0.89      0.88      2140



In [38]:
#@title Run evaluation with Linear SVM classifier:
from sklearn.svm import LinearSVC
evaluate_classifier(LinearSVC(),
                    document_term_matrix_train, document_term_matrix_test,
                    labels_train, labels_test)

Accuracy: 0.8710280373831776
              precision    recall  f1-score   support

           0       0.86      0.91      0.88      1154
           1       0.89      0.82      0.85       986

    accuracy                           0.87      2140
   macro avg       0.87      0.87      0.87      2140
weighted avg       0.87      0.87      0.87      2140



Note: You can change the meta-parameters of classifiers, e.g. avoid using Prior for MNB, penalty for regulazation in LogRegression, etc.
<br>The optimal meta-parameters are usually optimized on a separate tuning dataset using cross-validation
<br>(see [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) in scikit-learn or [Vizier](http://go/vizier) for Google internal optimization).

### Trying different features

In [39]:
# Check the results with n-grams.
ngram_vectorizer = CountVectorizer(ngram_range=(1,2))
document_ngram_matrix_train = ngram_vectorizer.fit_transform(corpus_train).toarray()
document_ngram_matrix_test = ngram_vectorizer.transform(corpus_test).toarray()

evaluate_classifier(LogisticRegression(), 
                    document_ngram_matrix_train, document_ngram_matrix_test,
                    labels_train, labels_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.8864485981308411
              precision    recall  f1-score   support

           0       0.85      0.95      0.90      1154
           1       0.93      0.81      0.87       986

    accuracy                           0.89      2140
   macro avg       0.89      0.88      0.88      2140
weighted avg       0.89      0.89      0.89      2140



In [40]:
# Check what happens when only features with high PMI are used (feature selection).
# Note: This evaluation setup is not fair as PMI relatedness scores were obtained over the full data (including test data). 
PMI_selected_words = [word for word, score in PMI_relatedness.items() if abs(score) >= 1.0]
filtered_vectorizer = CountVectorizer(vocabulary = PMI_selected_words)

document_filtered_term_matrix_train = filtered_vectorizer.fit_transform(corpus_train).toarray()
document_filtered_term_matrix_test = filtered_vectorizer.transform(corpus_test).toarray()

evaluate_classifier(LogisticRegression(), 
                    document_filtered_term_matrix_train, document_filtered_term_matrix_test, 
                    labels_train, labels_test)

Accuracy: 0.891588785046729
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      1154
           1       0.94      0.82      0.87       986

    accuracy                           0.89      2140
   macro avg       0.90      0.89      0.89      2140
weighted avg       0.90      0.89      0.89      2140



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Another classification problem: Sentiment Analysis

One common classification problem is to understand whether users-generated texts (e.g. reviews or general feedback) are rather positive or negative.
<br>The annotated data usually come from the reviews' websites, where users explicitly provide ratings along with the reviews.
<br>The area of [sentiment analysis](https://en.wikipedia.org/wiki/Sentiment_analysis) is large and involves understanding of
<br>various expressions of opinions, emotions, and attidudues. 

In this class, let's see how would you predict polarity of movie reviews. We will focus on re-using already existing methods,
<br>showing you how the classification would be actually done, without wasting time to reimplement what was already
<br>implemented.

## Exercise: Exploration of different classification approaches

*Exercise*: Go over the cells here. Make sure you understand what is happening. Check the results of various classification methods. 

### Data and function imports

In [41]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [ ]:
#    Imports
from nltk.corpus import movie_reviews

import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
#    Extract lists of ids of the reviews that are labeled as negative and positive
negative_ids = movie_reviews.fileids('neg')
positive_ids = movie_reviews.fileids('pos')

In [44]:
print ("---> Some examples of the document ids with the negative reviews.")
print (negative_ids[:5])

---> Some examples of the document ids with the negative reviews.
['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


In [ ]:
#    Let's prepare the list of texts and their classes as a training examples
#    Note: the texts are already preprocessed and split to tokens.
negative_reviews = [
    " ".join(movie_reviews.words(fileids=[f])) 
    for f in negative_ids
]
positive_reviews = [
    " ".join(movie_reviews.words(fileids=[f])) 
    for f in positive_ids
]

texts = negative_reviews + positive_reviews
labels = [0] * len(negative_reviews) + [1] * len(positive_reviews)

In [47]:
#    See example text.
print (texts[0])

plot : two teen couples go to a church party , drink and then drive . they get into an accident . one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . what ' s the deal ? watch the movie and " sorta " find out . . . critique : a mind - fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn ' t snag this one correctly . they seem to have taken this pretty neat concept , but executed it terribly . so what are the problems with the movie ? well , its main problem is that it ' s simply too jumbled . it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no idea

### Running basic classifiers

In [ ]:
#    Bulding the pipeline to transform texts and classify them.
def text_classifier(vectorizer, transformer, classifier):
  return Pipeline(
      [('vectorizer', vectorizer),
       ('transformer', transformer),
       ('classifier', classifier)]
  )

In [50]:
#    Let's estimate the quality of classification from various methods
for classifier in [MultinomialNB, LogisticRegression, LinearSVC, SGDClassifier]:
  print (classifier)
  print (cross_val_score(
            text_classifier(
                CountVectorizer(),
                TfidfTransformer(),
                classifier()), 
            texts,
            labels,
            cv = 5)
         .mean())

<class 'sklearn.naive_bayes.MultinomialNB'>
0.8125
<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8210000000000001
<class 'sklearn.svm.classes.LinearSVC'>
0.8545
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.8575000000000002


In [51]:
#    Now let's use the classifier trained on the whole data for the prediction
#    on some random texts.

classification_pipeline = Pipeline(
                            [("vectorizer", TfidfVectorizer()),
                             ("classifier", LinearSVC())]
                          )

classification_pipeline.fit(texts, labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
   

In [52]:
# Label 1 is for positive prediction, 0 - for negative one.
print (classification_pipeline.predict([
    "Amazing film! I will advice it to all my friends. Genious",
    "Awful film! The man who advised me to watch it is really crazy idiot.",
    "I did not find this movie so great!"]))

[1 0 0]


### Reducing dimentionality of feature space

In [53]:
# Computing number of all present terms:
vectorizer = CountVectorizer()
vectorizer.fit(texts)
len(vectorizer.get_feature_names())


39659

We can see that the matrix is quite large and sparse. We need to [reduce dimentionality](https://en.wikipedia.org/wiki/Dimensionality_reduction) (or in other words smoothen or select the features to reduce their number). Read here about common [dimentionality reduction methods](https://www.kdnuggets.com/2015/05/7-methods-data-dimensionality-reduction.html).

In [ ]:
from sklearn.decomposition import NMF, TruncatedSVD

v = CountVectorizer()
mx = v.fit_transform(texts)
mf = TruncatedSVD(10)
u = mf.fit_transform(mx)

In [55]:
for transform in [TruncatedSVD, NMF]:
    print (transform)
    print (cross_val_score(
              text_classifier(
                  CountVectorizer(),
                  transform(n_components=10),
                  LinearSVC()),
              texts,
              labels)
           .mean())
#    The results look way lower than with the simple methods we tried earlier.
#    What if we set n_components to something highers?

<class 'sklearn.decomposition.truncated_svd.TruncatedSVD'>
0.538983594372816
<class 'sklearn.decomposition.nmf.NMF'>
0.6430082777388167


In [56]:
#    Important: will take about 3 minutes :)
print (cross_val_score(
          text_classifier(
              CountVectorizer(),
              TruncatedSVD(n_components=1000),
              LinearSVC()),
          texts,
          labels)
       .mean())

0.8120171069272866


### Trying ensemble trees on top of reduced features

In [ ]:
#    Again some more imports
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [58]:
print (cross_val_score(
    Pipeline([
            ("vectorizer", CountVectorizer()),
            ("transformer", TruncatedSVD(100)),
            ("classifier", RandomForestClassifier(100))
        ]),
    texts,
    labels
    ).mean()
)

0.71951742161323


In [59]:
#    We can try more components and more trees :) 
#    Note: will run about 5 minutes.
print (cross_val_score(
         text_classifier(
             CountVectorizer(),
             TruncatedSVD(n_components=1000),
             RandomForestClassifier(1000)),
         texts,
         labels)
      .mean())

0.7130094166022309


### Trying with other initial features

In [60]:
#    Let's now see what we will get if we use TFIDF (TfidfVectorizer)
#    instead of words frequencies (CountVectorizer).
#    Note: will take about 3 minutes.
print (cross_val_score(
          text_classifier(
              TfidfVectorizer(),
              TruncatedSVD(n_components=1000),
              LinearSVC()),
          texts,
          labels)
       .mean())

0.8405201608794424


In [61]:
#    Note: will run about 5 minutes.
print (cross_val_score(
         text_classifier(
             TfidfVectorizer(),
             TruncatedSVD(n_components=1000),
             RandomForestClassifier(1000)),
         texts,
         labels)
      .mean())

0.5789936643230057


### Combining TFIDF and SVD features.

We combine TF-IDF features and the reduced-dimentionality representation by SVD. In contrast with above transformation, here both sparse (TF-IDF of terms) and dense (SVD representation) features will be present.

In [ ]:
#    Import Feature union
from sklearn.pipeline import FeatureUnion

estimators = [('tfidf', TfidfTransformer()), ('svd', TruncatedSVD(100))]
combined = FeatureUnion(estimators)

In [65]:
print (cross_val_score(
          Pipeline([
              ("vectorizer", CountVectorizer()),
              ("transformer", combined),
              ("classifier", LinearSVC())
              ]),
          texts,
         labels)
      .mean())

0.7870145594696493


# Word Embeddings

### Description

[Word embeddings](https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795) is a set of language modeling and feature learning techniques in
<br>NLP where words/phrases/sentences/paragraphs/documents are mapped to vectors of real numbers (usually of less dimentilnality).

The idea about this form of dimentionality reduction is to capture semantic/morphological/contextual/hierarchical/etc
<br>information as possible from the original text. While training the models to find the embeddings, several directions could be taken:
<br>(1) preserving the [morphological structure](https://arxiv.org/pdf/1607.04606.pdf) (subword information, etc.);
<br>(2) [word context](https://arxiv.org/pdf/1411.2738.pdf) representation;
<br>(3) [global corpus statistics](https://nlp.stanford.edu/pubs/glove.pdf);
<br>(4) [word hierarchy](https://arxiv.org/pdf/1705.08039.pdf) as in WordNet;
<br>(5) [relationship between documents](https://nlp.stanford.edu/IR-book/html/htmledition/latent-semantic-indexing-1.html) and the terms they contain.

* **CBOW**

An approach where we train a model to predict one word given another word (given a set of words in the context).
<br>$ p(w_i | w_j) $. To do so, we train a neural network with the following structure:

input (one-hot encoded vectors for the words, V x 1; where V is the size of the vocabulary) -> 

hidden layer (weight matrix W, V x N; where N is the size of the hidden layer) ->

output (weight matrix W', N x V) + softmax

<img src="https://cdn-images-1.medium.com/max/1600/1*6OSkLBIxnXOwloUozZljjw.png" width="40%">

* **Skip-gram**

Opposite to the previous version, we try to predict the context words by a target word in the input
<br>$ \sum_{all\ words\ i} \sum_{j - k}^{j + k} p( w_{j}| w_i) $,
<br>where $k$ is the window of context we are trying to predic with a given target workd $w_i$.

<img src="https://cdn-images-1.medium.com/max/1600/1*pRV0o3uuY7n5NdHhAaAzlw.png" width="40%">

* **[Glove](https://nlp.stanford.edu/pubs/glove.pdf)**

This approach tries to capture the meaning of one word embedding with the structure of the whole observed corpus.
<br>This model is trained on the global co-occurrence counts and uses the word statistics.
<br>More details and explations are either in the paper or in [this tutorial](https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795).

* **FastText**

FastText is basically skip-gram model with negative sampling (trying to max the probability of the actual context words and
<br>minimize the probability of random words taken from the dictionary) but also we can add the internal structure of words
<br>by splitting them into the bag of characters n-grams.

Fast and easy to start with direction, if you need to train the embeddings on your own corpus:
<br>Embeddings can be trained on your provided corpus, e.g. using [FastText](https://github.com/facebookresearch/fastText).
([Wikipedia dumps](https://dumps.wikimedia.org/) could be of use)

### Pretrained Word Embeddings

#### Open-source, external embeddings

[Fasttext models](https://fasttext.cc/docs/en/english-vectors.html):
* [crawl-300d-2M.vec.zip](https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip): 2 million word vectors trained on Common Crawl (600B tokens).
* [wiki-news-300d-1M.vec.zip](https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip): 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens).
* [wiki-news-300d-1M-subword.vec.zip](https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M-subword.vec.zip): 1 million word vectors trained with subword infomation on Wikipedia 2017,
<br>UMBC webbase corpus and statmt.org news dataset (16B tokens).
* [Wiki word vectors](https://fasttext.cc/docs/en/pretrained-vectors.html), dim=300: [wiki.en.zip](https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.zip): bin+text model

[Google Word2Vec](https://code.google.com/archive/p/word2vec/)
* Pretrained word/phrase vectors:
  * [GoogleNews-vectors-negative300.bin.gz](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)
  * [GoogleNews-vectors-negative300-SLIM.bin.gz](https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz): slim version with app. 300k words
* Pretrained entity vectors:
  * [freebase-vectors-skipgram1000.bin.gz](https://docs.google.com/file/d/0B7XkCwpI5KDYaDBDQm1tZGNDRHc/edit?usp=sharing): Entity vectors trained on 100B words from various news articles
  * [freebase-vectors-skipgram1000-en.bin.gz](https://docs.google.com/file/d/0B7XkCwpI5KDYeFdmcVltWkhtbmM/edit?usp=sharing): Entity vectors trained on 100B words from various news articles,
  <br>using the deprecated /en/ naming (more easily readable); the vectors are sorted by frequency

[GloVe](https://nlp.stanford.edu/projects/glove/): Global Vectors for Word Representation
* [glove.6B.zip](http://nlp.stanford.edu/data/glove.6B.zip): Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, 822 MB download).
<br>Here's an example in action. TODO: Add a link.
* [glove.840B.300d.zip](http://nlp.stanford.edu/data/glove.840B.300d.zip): Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download)

Let's load some pretrained embeddings.

In [ ]:
#@title [Fix it if you can later] Some potential help to load the model from drive

import gensim
from gensim.models.keyedvectors import KeyedVectors
from google.colab import auth
from io import BytesIO
from oauth2client.client import GoogleCredentials
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import shutil
import smart_open
from tempfile import NamedTemporaryFile
from zipfile import ZipFile

def authentication():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive

def read_zipfile(input_file_id='glove.6B.zip'):
  drive = authentication()

  file_id = drive.ListFile({'q':"title='"+input_file_id+"'"}).GetList()[0]['id']
  f = drive.CreateFile({'id': file_id})
  zf_string = f.GetContentString(encoding='cp862')
  zf_bytes = BytesIO(zf_string.encode('cp862'))
  return ZipFile(zf_bytes, 'r')

def read_files_zipfile(zf):
  _files = []
  _file = {}
  for file_name in zf.namelist():
    _file['name'] = file_name
    _file['data'] = zf.read(file_name)
    _files.append(_file)
    _file = {}
  return _files

def get_lines(zipfile, file_name):
  with zipfile.open(file_name, 'r') as f:
    num_lines = sum(1 for line in f) - 1
  with zipfile.open(file_name, 'r') as f:
    num_dims = len(f.readline().split()) - 1
  return num_lines, num_dims

def prepend_slow(zipfile, file_name, line):
  tmp_file_name = None
  with zf.open(file_name, 'r') as fin:
    with NamedTemporaryFile(delete=False) as tmp_file:
      tmp_file.write(gensim_first_line + b'\n')
      for gensim_first_line in fin:
        tmp_file.write(gensim_first_line)
      tmp_file_name = tmp_file.name

  if tmp_file_name is not None:
    return tmp_file_name
  
def drive_upload_content_file(drive, file_name, tmp_file_path, parent_id, verbose=True):
  metadata = {'title': file_name}
  if parent_id:
    metadata['parents'] = [{'kind': 'drive#fileLink', 'id': parent_id}]
  uploaded = drive.CreateFile(metadata)
  uploaded.SetContentFile(tmp_file_path)
  uploaded.Upload()
  if verbose:
    print('Uploaded file with ID {}'.format(uploaded.get('id')))

def glove_to_word2vec(zipfile, file_name, **kwargs):
  num_lines, num_dims = get_lines(zipfile, file_name)
  gensim_first_line = '{} {}'.format(num_lines, num_dims)
  gensim_first_line = gensim_first_line.encode()
  tmp_file_name = prepend_slow(zipfile, file_name, gensim_first_line)
  print (tmp_file_name)
  
  verbose = kwargs.get('verbose', True)
  parent_id = kwargs.get('parent_id', None)
  
  if tmp_file_name is not None:
    drive = authentication()
    drive_upload_content_file(drive, file_name, tmp_file_name, parent_id, verbose)
    
# zf = read_zipfile('glove.6B.zip')
# print ([f['name'] for f in read_files_zipfile(zf)])

# glove_to_word2vec(zf, 'glove.6B/glove.6B.200d.txt')

#### Loading embeddings

In [66]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.0)


In [67]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

#    Note: it might take several minutes. Be patient!
from gensim.scripts.glove2word2vec import glove2word2vec

glove2word2vec(glove_input_file='glove.6B.50d.txt',
               word2vec_output_file="gensim_glove_vectors.txt")


--2019-07-23 14:21:41--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-07-23 14:21:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-07-23 14:21:42--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

(400000, 50)

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt")


In [69]:
model.similar_by_vector(model["madonna"] - model["singer"] + model["painter"])

[('caravaggio', 0.7772032618522644),
 ('michelangelo', 0.7727645039558411),
 ('painting', 0.7709366083145142),
 ('rembrandt', 0.745723307132721),
 ('titian', 0.742861807346344),
 ('gogh', 0.7370840311050415),
 ('picasso', 0.7252718806266785),
 ('painter', 0.7237308025360107),
 ('altarpiece', 0.7143588662147522),
 ('monet', 0.7027225494384766)]

In [70]:
model.similar_by_vector(model["obama"] - model["usa"] + model["japan"])

[('koizumi', 0.8049983978271484),
 ('hashimoto', 0.7850980758666992),
 ('putin', 0.7768493294715881),
 ('obama', 0.7716479301452637),
 ('barack', 0.747847318649292),
 ('bush', 0.7318081259727478),
 ('junichiro', 0.7254975438117981),
 ('clinton', 0.7190637588500977),
 ('ryutaro', 0.7096714973449707),
 ('stimulus', 0.7046528458595276)]

In [71]:
#@title Find closest words to the input word
input_words = ['earthquake', 'happy', 'i', 'queen']

for word in input_words:
  print ("---\n", word, "\n---")
  for similar_word in model.most_similar(word):
    print (similar_word)


---
 earthquake 
---
('quake', 0.9659477472305298)
('temblor', 0.8302474021911621)
('tsunami', 0.825542688369751)
('tremor', 0.822434663772583)
('magnitude', 0.7913191318511963)
('disaster', 0.7809879183769226)
('earthquakes', 0.7694811820983887)
('epicenter', 0.7664315700531006)
('jolted', 0.7571674585342407)
('aftershock', 0.7534422278404236)
---
 happy 
---
("'m", 0.9142324328422546)
('everyone', 0.8976402878761292)
('everybody', 0.8965489864349365)
('really', 0.8839760422706604)
('me', 0.8784631490707397)
('definitely', 0.8762789368629456)
('maybe', 0.875670313835144)
("'d", 0.8718012571334839)
('feel', 0.8707678318023682)
('i', 0.8707453608512878)
---
 i 
---
("'d", 0.9566037654876709)
('me', 0.9546188116073608)
('maybe', 0.930670440196991)
('know', 0.927360475063324)
("n't", 0.9259788393974304)
('you', 0.9237291812896729)
("'m", 0.9199258089065552)
('else', 0.9198518395423889)
('never', 0.9197366237640381)
('really', 0.9190417528152466)
---
 queen 
---
('princess', 0.851516604423

Nice visualization of some pretrained embeddings: https://projector.tensorflow.org/

## Exercise: Use word embeddings in sentiment analysis representation

*Exercise*: Define your own vectorizer based on the pretrained word embeddings. Compare the evaluation results with the previous methods.

Type your solution in the cell below: 

Hint: See in the preprocessing part for how to represent documents using given embeddings.

### <font color="grey">Exercise solution:  Use word embeddings in sentiment analysis representation </font>

In [ ]:
# Represent documents as mean of embeddings.
import numpy as np
num_dimension = model.vector_size  # Get dimensionality out of the model.
document_embeddings = []
for document in texts:
  document_word_embeddings = \
      [model[token] for token in document if token in model]
  if (len(document_word_embeddings) > 0):
    document_embeddings.append(np.mean(document_word_embeddings, axis=0))
  else:
    document_embeddings.append(np.zeros(num_dimension))

In [ ]:
import lightgbm
lgb = lightgbm.LGBMClassifier

In [79]:
for classifier in [LogisticRegression, LinearSVC, RandomForestClassifier, lgb]:
  print (classifier)
  print (cross_val_score(
            classifier(), 
            document_embeddings,
            labels,
            cv = 5)
         .mean())

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.5875
<class 'sklearn.svm.classes.LinearSVC'>
0.5985
<class 'sklearn.ensemble.forest.RandomForestClassifier'>
0.55
<class 'lightgbm.sklearn.LGBMClassifier'>
0.5865


## *Challenges of sentiment analysis*

Though we have simply applied some transformations that we have studies, there is usually more than that to each particular problem.
<br>While for some problems Bag-Of-Words (BOW) representations could work pretty well,
<br>some problems might require more specific preprocessing, data representation, etc.

For example, in case of sentiment analysis the following phenomena should be treated:
1. Negation
<br>
Plain BOW representation -  \["not", "hate"\] vs. \["hate"\] - might seem to be quite similar, whereas they represent completely opposite polarities.
<br>Usually, some rule based approaches could be used on the preprocessing or data representation step
<br>(like creating syntactic negated term ('NOT_hate') or merge all negations to the same bucket/feature).
2. Modulated/contextual polarity
<br>
Some words could have completely different meanings and polarity in different contexts.
<br>E.g., "high resolution" is rather positive, while "high price" is usually negative one.
<br>Of course, such information is never stated explicitly and rather is common sense knowledge to us.
3. Domain specificity
<br>
Some expressions might be either positive and negative in different domains.
<br>For example, in the teenage movie review swearing words might be considered positive, while,
<br>for the documentary such words are either out of place or mostly negative.
<br>Or, depending on the news source, 'education abroad' could be negative for the country
<br>(as people tend to stay abroad afterwards), or rather positive in the context of educational opportunities, etc.



## More advanced techniques for sentiment analysis

There already exist some deep learning solutions with great accuracy in predicting polarity of texts.
<br>You can even try and see if such system can catch your sentiments.
<br>
One of the approaches that outperforms "old-school" feature/rule based systems is based on the [reccurent neural network](http://en.wikipedia.org/wiki/Recurrent_neural_network) and made in [Stanford](https://nlp.stanford.edu/sentiment/).

Let's try the [live demo](http://nlp.stanford.edu:8080/sentiment/rntnDemo.html).

# ---